In [1]:
import os
import glob
import json
import sys
import cv2
import math
import datetime
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import scipy.signal
import shutil
import torch.optim as optim
import operator
import matplotlib
matplotlib.use('Agg')

from matplotlib import pyplot as plt
from copy import deepcopy
from functools import partial
from torchvision.ops import nms
from PIL import Image
from tqdm import tqdm
from torch.utils.data import DataLoader

from dataset import YoloDataset
from yolov8m_Swin_transfomer import YoloBody
from utils import get_lr_scheduler,set_optimizer_lr,get_classes,yolo_dataset_collate,get_lr
from class_utils import DecodeBox,EvalCallback,BboxLoss,TaskAlignedAssigner,Loss,ModelEMA,LossHistory

In [2]:
if __name__ == "__main__":
    #---------------------------------#
    #   Cuda    是否使用Cuda
    #           没有GPU可以设置成False
    #---------------------------------#
    Cuda            = True
    #---------------------------------------------------------------------#
    #   classes_path    指向VOCdata下的txt，与自己训练的数据集相关 
    #---------------------------------------------------------------------#
    classes_path    = 'VOCdata/classes.txt'
    
    #------------------------------------------------------#
    #   input_shape     输入的shape大小，一定要是32的倍数
    #------------------------------------------------------#
    input_shape     = [640, 640]
    
    #------------------------------------------------------#
    #   phi             所使用到的yolov8的版本
    #                   n : 对应yolov8_n
    #                   s : 对应yolov8_s
    #                   m : 对应yolov8_m
    #                   l : 对应yolov8_l
    #                   x : 对应yolov8_x
    #------------------------------------------------------#
    phi             = 'm'
    
    #------------------------------------------------------------------#
    #   mosaic              马赛克数据增强。
    #   mosaic_prob         每个step有多少概率使用mosaic数据增强，默认50%。
    #
    #   mixup               是否使用mixup数据增强，仅在mosaic=True时有效。
    #                       只会对mosaic增强后的图片进行mixup的处理。
    #   mixup_prob          有多少概率在mosaic后使用mixup数据增强，默认50%。
    #                       总的mixup概率为mosaic_prob * mixup_prob。
    #------------------------------------------------------------------#
    mosaic              = True
    mosaic_prob         = 0.5
    mixup               = True
    mixup_prob          = 0.5

    #------------------------------------------------------------------#
    #   解冻阶段训练参数
    #   此时模型的主干不被冻结了，特征提取网络会发生改变
    #   占用的显存较大，网络所有的参数都会发生改变
    #   Epoch          模型总共训练的轮次
    #   batch_size     模型训练批次的大小
    #------------------------------------------------------------------#
    Epoch      = 100
    batch_size= 16

    #------------------------------------------------------------------#
    #   Init_lr         模型的最大学习率
    #   Min_lr          模型的最小学习率，默认为最大学习率的0.01
    #------------------------------------------------------------------#
    Init_lr             = 1e-2
    Min_lr              = Init_lr * 0.01
    #------------------------------------------------------------------#
    #   momentum        优化器内部使用到的momentum参数
    #   weight_decay    权值衰减，可防止过拟合
    #                   adam会导致weight_decay错误，使用adam时建议设置为0。
    #------------------------------------------------------------------#
    momentum            = 0.937
    weight_decay        = 0
    
    #------------------------------------------------------------------#
    #   save_period     多少个epoch保存一次权值
    #------------------------------------------------------------------#
    save_period         = 10
    
    #------------------------------------------------------------------#
    #   save_dir        权值与日志文件保存的文件夹
    #------------------------------------------------------------------#
    save_dir            = 'logs'
    
    #------------------------------------------------------------------#
    #   eval_flag       是否在训练时进行评估，评估对象为验证集
    #   eval_period     代表多少个epoch评估一次，不建议频繁的评估
    #                   评估需要消耗较多的时间，频繁评估会导致训练非常慢
    #   此处获得的mAP会与get_map.py获得的会有所不同，原因有二：
    #   （一）此处获得的mAP为验证集的mAP。
    #   （二）此处设置评估参数较为保守，目的是加快评估速度。
    #------------------------------------------------------------------#
    eval_flag           = True
    eval_period         = 10
    
    #------------------------------------------------------------------#
    #   num_workers     用于设置是否使用多线程读取数据
    #                   开启后会加快数据读取速度，但是会占用更多内存
    #------------------------------------------------------------------#
    num_workers         = 4

    #------------------------------------------------------#
    #   train_annotation_path   训练图片路径和标签
    #   val_annotation_path     验证图片路径和标签
    #------------------------------------------------------#
    train_annotation_path   = './VOCdata/2012_train.txt'
    val_annotation_path     = './VOCdata/2012_val.txt'

    #------------------------------------------------------#
    #   获取classes和anchor
    #------------------------------------------------------#
    class_names, num_classes = get_classes(classes_path)

    #------------------------------------------------------#
    #   创建yolo模型
    #------------------------------------------------------#
    model = YoloBody(input_shape, num_classes, phi,pretrained=False)

    #------------------------------------------------------#
    #   设置用到的显卡
    #------------------------------------------------------#
    device          = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    local_rank      = 0
    
    #----------------------#
    #   使用显卡训练
    #----------------------#
    model_train     = model.train()
    if Cuda:
        model_train = torch.nn.DataParallel(model)
        cudnn.benchmark = True
        model_train = model_train.cuda()
    
    #----------------------#
    #   获得损失函数
    #----------------------#
    yolo_loss = Loss(model)
    
    #----------------------#
    #   是否对dataset和dataloader数据进行打乱处理
    #----------------------#
    shuffle         = True
    
    #----------------------#
    #   记录Loss
    #----------------------#
    time_str        = datetime.datetime.strftime(datetime.datetime.now(),'%Y_%m_%d_%H_%M_%S')
    log_dir         = os.path.join(save_dir, "loss_" + str(time_str))
    loss_history    = LossHistory(log_dir, model, input_shape=input_shape)

    #----------------------------#
    #   权值平滑
    #----------------------------#
    ema = ModelEMA(model_train)

    #-------------------------------------------------------------------#
    #   判断当前batch_size，自适应调整学习率
    #-------------------------------------------------------------------#
    nbs             = batch_size
    lr_limit_max    = 1e-3
    lr_limit_min    = 3e-4
    Init_lr_fit     = min(max(batch_size / nbs * Init_lr, lr_limit_min), lr_limit_max)
    Min_lr_fit      = min(max(batch_size / nbs * Min_lr, lr_limit_min * 1e-2), lr_limit_max * 1e-2)
    lr_scheduler_func = get_lr_scheduler(Init_lr_fit, Min_lr_fit, Epoch)
    
    #---------------------------#
    #   读取数据集对应的txt
    #---------------------------#
    with open(train_annotation_path, encoding='utf-8') as f:
        train_lines = f.readlines()
    with open(val_annotation_path, encoding='utf-8') as f:
        val_lines   = f.readlines()
    num_train   = len(train_lines)
    num_val     = len(val_lines)
    
    #---------------------------------------#
    #   定义优化器
    #---------------------------------------#     
    pg0, pg1, pg2 = [], [], []  
    for k, v in model.named_modules():
        if hasattr(v, "bias") and isinstance(v.bias, nn.Parameter):
            pg2.append(v.bias)    
        if isinstance(v, nn.BatchNorm2d) or "bn" in k:
            pg0.append(v.weight)    
        elif hasattr(v, "weight") and isinstance(v.weight, nn.Parameter):
            pg1.append(v.weight)   
    optimizer =optim.Adam(pg0, Init_lr_fit, betas = (momentum, 0.999))
    optimizer.add_param_group({"params": pg1, "weight_decay": weight_decay})
    optimizer.add_param_group({"params": pg2})


    #---------------------------------------#
    #   判断每一个epoch的长度
    #---------------------------------------#
    epoch_step      = num_train // batch_size
    epoch_step_val  = num_val // batch_size

    #---------------------------------------#
    #   构建数据集加载器。
    #---------------------------------------#
    train_dataset   = YoloDataset(train_lines, input_shape, num_classes,\
                                    mosaic=mosaic, mixup=mixup, mosaic_prob=mosaic_prob, mixup_prob=mixup_prob, train=True)
    val_dataset     = YoloDataset(val_lines, input_shape, num_classes,\
                                    mosaic=False, mixup=False, mosaic_prob=0, mixup_prob=0, train=False)

    gen             = DataLoader(train_dataset, shuffle = shuffle, batch_size = batch_size, num_workers = num_workers, pin_memory=True,
                                drop_last=True, collate_fn=yolo_dataset_collate)
    gen_val         = DataLoader(val_dataset  , shuffle = shuffle, batch_size = batch_size, num_workers = num_workers, pin_memory=True, 
                                drop_last=True, collate_fn=yolo_dataset_collate)

    #----------------------#
    #   记录eval的map曲线
    #----------------------#
    eval_callback   = EvalCallback(model, input_shape, class_names, num_classes, val_lines, log_dir, Cuda, \
                                    eval_flag=eval_flag, period=eval_period)




/root/miniconda3/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


initialize network with normal type


In [3]:
    if True:
        #---------------------------------------#
        #   开始模型训练
        #---------------------------------------#
        for epoch in range (Epoch):

            
            gen.dataset.epoch_now       = epoch
            gen_val.dataset.epoch_now   = epoch


            set_optimizer_lr(optimizer, lr_scheduler_func, epoch)
            loss        = 0
            val_loss    = 0
        
            if local_rank == 0:
                print('Start Train')
                pbar = tqdm(total=epoch_step,desc=f'Epoch {epoch + 1}/{Epoch}',postfix=dict,mininterval=0.3)
            model_train.train()
            for iteration, batch in enumerate(gen):
                if iteration >= epoch_step:
                    break
        
                images, bboxes = batch
                with torch.no_grad():
                    if Cuda:
                        images = images.cuda(local_rank)
                        bboxes = bboxes.cuda(local_rank)
                #----------------------#
                #   清零梯度
                #----------------------#
                optimizer.zero_grad()

                #----------------------#
                #   前向传播
                #----------------------#
                # dbox, cls, origin_cls, anchors, strides 
                outputs = model_train(images)
                loss_value = yolo_loss(outputs, bboxes)
                #----------------------#
                #   反向传播
                #----------------------#
                loss_value.backward()
                torch.nn.utils.clip_grad_norm_(model_train.parameters(), max_norm=10.0)  # clip gradients
                optimizer.step()

                if ema:
                    ema.update(model_train)
        
                loss += loss_value.item()
                
                if local_rank == 0:
                    pbar.set_postfix(**{'loss'  : loss / (iteration + 1), 
                                        'lr'    : get_lr(optimizer)})
                    pbar.update(1)
        
            if local_rank == 0:
                pbar.close()
                print('Finish Train')
                print('Start Validation')
                pbar = tqdm(total=epoch_step_val, desc=f'Epoch {epoch + 1}/{Epoch}',postfix=dict,mininterval=0.3)
        
            if ema:
                model_train_eval = ema.ema
            else:
                model_train_eval = model_train.eval()
                
            for iteration, batch in enumerate(gen_val):
                if iteration >= epoch_step_val:
                    break
                images, bboxes = batch[0], batch[1]
                with torch.no_grad():
                    if Cuda:
                        images = images.cuda(local_rank)
                        bboxes = bboxes.cuda(local_rank)
                    #----------------------#
                    #   清零梯度
                    #----------------------#
                    optimizer.zero_grad()
                    #----------------------#
                    #   前向传播
                    #----------------------#
                    outputs     = model_train_eval(images)
                    loss_value  = yolo_loss(outputs, bboxes)
        
                val_loss += loss_value.item()
                if local_rank == 0:
                    pbar.set_postfix(**{'val_loss': val_loss / (iteration + 1)})
                    pbar.update(1)

            if local_rank == 0:
                pbar.close()
                print('Finish Validation')
                #记录训练和验证的损失值
                loss_history.append_loss(epoch + 1, loss / epoch_step, val_loss / epoch_step_val)
                #对模型进行评估，记录map的值
                eval_callback.on_epoch_end(epoch + 1, model_train_eval)
                print('Epoch:'+ str(epoch + 1) + '/' + str(Epoch))
                print('Total Loss: %.3f || Val Loss: %.3f ' % (loss / epoch_step, val_loss / epoch_step_val))
                
                #-----------------------------------------------#
                #   保存权值
                #-----------------------------------------------#
                if ema:
                    save_state_dict = ema.ema.state_dict()
                else:
                    save_state_dict = model.state_dict()
        
                if (epoch + 1) % save_period == 0 or epoch + 1 == Epoch:
                    torch.save(save_state_dict, os.path.join(save_dir, "ep%03d-loss%.3f-val_loss%.3f.pth" % (epoch + 1, loss / epoch_step, val_loss / epoch_step_val)))
                    
                if len(loss_history.val_loss) <= 1 or (val_loss / epoch_step_val) <= min(loss_history.val_loss):
                    print('Save best model to best_epoch_weights.pth')
                    torch.save(save_state_dict, os.path.join(save_dir, "best_epoch_weights.pth"))
                    
                torch.save(save_state_dict, os.path.join(save_dir, "last_epoch_weights.pth"))

Start Train


Epoch 1/100:   0%|          | 0/866 [00:00<?, ?it/s<class 'dict'>]/root/Untitled Folder/utils.py:862: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b1_x1, b1_y1, b1_x2, b1_y2 = torch.chunk(torch.tensor(box1), 4, dim=-1)
/root/Untitled Folder/utils.py:863: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b2_x1, b2_y1, b2_x2, b2_y2 = torch.chunk(torch.tensor(box2), 4, dim=-1)
Epoch 1/100: 100%|██████████| 866/866 [03:11<00:00,  4.53it/s, loss=98.3, lr=0.001]   


Finish Train
Start Validation


Epoch 1/100: 100%|██████████| 96/96 [00:08<00:00, 11.16it/s, val_loss=11.9]


Finish Validation
Epoch:1/100
Total Loss: 98.349 || Val Loss: 11.927 
Save best model to best_epoch_weights.pth
Start Train


...

Finish Train
Start Validation


Epoch 100/100: 100%|██████████| 96/96 [00:08<00:00, 11.48it/s, val_loss=5]   


Finish Validation
Get map.


100%|██████████| 1542/1542 [00:42<00:00, 36.68it/s]


Calculate Map.
63.72% = aeroplane AP 	||	score_threhold=0.5 : F1=0.66 ; Recall=52.05% ; Precision=88.37%
36.87% = bicycle AP 	||	score_threhold=0.5 : F1=0.37 ; Recall=25.00% ; Precision=69.57%
35.69% = bird AP 	||	score_threhold=0.5 : F1=0.35 ; Recall=22.50% ; Precision=81.82%
16.04% = boat AP 	||	score_threhold=0.5 : F1=0.05 ; Recall=2.78% ; Precision=66.67%
7.38% = bottle AP 	||	score_threhold=0.5 : F1=0.09 ; Recall=4.68% ; Precision=72.73%
62.18% = bus AP 	||	score_threhold=0.5 : F1=0.67 ; Recall=51.61% ; Precision=96.97%
51.27% = car AP 	||	score_threhold=0.5 : F1=0.50 ; Recall=34.41% ; Precision=91.43%
46.66% = cat AP 	||	score_threhold=0.5 : F1=0.38 ; Recall=25.00% ; Precision=78.95%
20.31% = chair AP 	||	score_threhold=0.5 : F1=0.10 ; Recall=5.18% ; Precision=61.90%
13.81% = cow AP 	||	score_threhold=0.5 : F1=0.00 ; Recall=0.00% ; Precision=0.00%
23.35% = diningtable AP 	||	score_threhold=0.5 : F1=0.18 ; Recall=10.94% ; Precision=58.33%
41.96% = dog AP 	||	score_threhold=0.5 : F